In [3]:
import pandas as pd
import numpy as np

In [4]:
x_train = pd.read_csv('WaterPumpData/features_train.csv')
y_train = pd.read_csv('WaterPumpData/labels_train.csv')

In [5]:
y_train['label'] = y_train.status_group.apply(lambda x: 1 if x == 'functional' else 0)

In [6]:
funder = x_train.funder.str.lower().value_counts()
funder = funder[funder > 250]
x_train['funder_mod'] = [i if i in funder else 'Unknown' for i in x_train.funder]

installer = x_train.funder.str.lower().value_counts()
installer = funder[funder > 250]
installer
x_train['installer_mod'] = [i if i in installer else 'Unknown' for i in x_train.installer]

In [7]:
import random
from random import sample

validation_set = sample(list(x_train.id), len(x_train)//5)

In [8]:
xx_train = x_train.loc[~x_train.id.isin(validation_set)]
xx_validate = x_train.loc[x_train.id.isin(validation_set)]
yy_train = y_train.loc[~y_train.id.isin(validation_set)].label
yy_validate = y_train.loc[y_train.id.isin(validation_set)].label

In [9]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

def pipeline(feature_set, numeric_set):
    numeric_features = numeric_set
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())])

    categorical_features =  [i for i in feature_set if i not in numeric_set]
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features),
            ('num', numeric_transformer, numeric_features)
        ])

    clf = Pipeline(steps=[('preprocessor', preprocessor)
                         ])

    clf.fit(xx_train)
    
    return clf

In [12]:
feature_set_8 = ['amount_tsh', 'population', 'funder_mod', 'district_code', 'gps_height', 'region', 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type_group', 'management_group', 'payment_type', 'quality_group', 'quantity_group', 'source_type', 'waterpoint_type_group']
numeric_features = ['amount_tsh', 'population', 'gps_height', 'construction_year']
clf = pipeline(feature_set_8, numeric_features)

xxx_train = pd.DataFrame(clf.transform(xx_train).todense())
xxx_validate = pd.DataFrame(clf.transform(xx_validate).todense())
yyy_train = pd.DataFrame(yy_train)
yyy_validate = pd.DataFrame(yy_validate)

In [13]:
from sklearn import preprocessing
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

def create_gradientbooster_model(feature_set, cat_var):
    clf = HistGradientBoostingClassifier()
    clf.fit(xxx_train, yy_train)
    
    return clf, xxx_validate, pd.DataFrame(yy_validate)

In [14]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
def create_randomforest_model(feature_set, cat_var):
    clf = RandomForestClassifier()
    clf.fit(xxx_train, yyy_train)
    
    return clf

In [15]:
grad_boost_model = HistGradientBoostingClassifier()
grad_boost_model.fit(xxx_train, yyy_train)

/Users/prateekagarwal2/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


HistGradientBoostingClassifier()

In [16]:
random_forest_model = RandomForestClassifier()
random_forest_model.fit(xxx_train, yyy_train)

/Users/prateekagarwal2/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier()

In [17]:
from keras.models import Sequential, Model
from keras.layers import Dense, concatenate, Input
from tensorflow.keras.optimizers import Adagrad, Adam
from keras.callbacks import EarlyStopping

ip = Input(shape=xxx_train.shape[1], name='ip')
x1 = Dense(250, activation='relu')(ip)
x2 = Dense(100, activation='relu')(x1)
x3 = Dense(50, activation='relu')(x2)
x4 = Dense(10, activation='relu')(x3)
output = Dense(1, activation='sigmoid')(x4)

nn_model = Model(inputs = ip, outputs=output)
nn_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
nn_model.fit(xxx_train, yyy_train, epochs=25, batch_size = 16)

Epoch 1/25
2970/2970 [==============================] - 3s 910us/step - loss: 0.4885 - accuracy: 0.7554
Epoch 2/25
2970/2970 [==============================] - 3s 905us/step - loss: 0.4481 - accuracy: 0.7822
Epoch 3/25
2970/2970 [==============================] - 3s 895us/step - loss: 0.4302 - accuracy: 0.7922
Epoch 4/25
2970/2970 [==============================] - 3s 898us/step - loss: 0.4191 - accuracy: 0.7987
Epoch 5/25
2970/2970 [==============================] - 3s 897us/step - loss: 0.4105 - accuracy: 0.8023
Epoch 6/25
2970/2970 [==============================] - 3s 997us/step - loss: 0.4023 - accuracy: 0.8080
Epoch 7/25
2970/2970 [==============================] - 3s 920us/step - loss: 0.3957 - accuracy: 0.8116
Epoch 8/25
2970/2970 [==============================] - 3s 937us/step - loss: 0.3898 - accuracy: 0.8145
Epoch 9/25
2970/2970 [==============================] - 3s 908us/step - loss: 0.3836 - accuracy: 0.8186
Epoch 10/25
2970/2970 [==============================] - 3s 911u

In [18]:
grad_boost_pred = grad_boost_model.predict(xxx_validate)
random_forest_pred = random_forest_model.predict(xxx_validate)
nn_pred = nn_model.predict(xxx_validate)

In [19]:
yyy_validate['gb_pred'] = grad_boost_pred
yyy_validate['rf_pred'] = random_forest_pred
yyy_validate['nn_pred_raw'] = nn_pred

In [20]:
yyy_validate['nn_pred'] = round(yyy_validate.nn_pred_raw)

In [21]:
yyy_validate["sum"] = yyy_validate.gb_pred + yyy_validate.rf_pred + yyy_validate.nn_pred

In [22]:
yyy_validate["consensus"] = yyy_validate['sum'].apply(lambda x: 1 if x in [2, 3] else 0)

In [23]:
(yyy_validate.consensus == yyy_validate.label).value_counts()

True     9639
False    2241
dtype: int64

In [24]:
(yyy_validate.rf_pred == yyy_validate.label).value_counts()

True     9545
False    2335
dtype: int64